In [ ]:
!nvidia-smi

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
import torch
device= "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
from datasets import load_dataset
dataset_samsum=load_dataset("knkarthick/samsum")

In [ ]:
dataset_samsum

In [ ]:
dataset_samsum["train"][0]["dialogue"]

In [ ]:
dataset_samsum["train"][0]["summary"]

In [ ]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f"split_lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}\n")
print(f"dialogue: {dataset_samsum["train"][0]["dialogue"]}\n")
print(f"summarization: {dataset_samsum["train"][0]["summary"]}")

In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)

  return {
      "input_ids":input_encodings["input_ids"],
      "attention_mask":input_encodings["attention_mask"],
      "labels":target_encodings["input_ids"]
  }


In [ ]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched = True)

In [ ]:
dataset_samsum_pt

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [ ]:
from transformers import TrainingArguments,Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=3,
    warmup_steps=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=1,
    fp16=True,
    report_to="none"
)


In [ ]:
trainer=Trainer(model=model_pegasus,args=trainer_args,
                tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                train_dataset=dataset_samsum_pt['test'],
                eval_dataset=dataset_samsum_pt['validation'])

In [ ]:
trainer.train()

In [ ]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

In [ ]:
from transformers import pipeline
gen_kwargs = {"length_penalty":0.8,"num_beams":8,"max_length":150}

pipe=pipeline("summarization",model="pegasus-samsum-model",tokenizer=tokenizer)

print("input")
sample_text="""Alex: Hey, did you guys finish the presentation slides for tomorrow’s client meeting?

Priya: Almost! I just need to add the market analysis section and update the charts with last week’s data.

Rahul: I updated the charts yesterday, Priya. Check the shared folder — file name is Market_Trends_v3.

Priya: Oh great, thanks Rahul! I was about to redo them. That saves me a lot of time.

Alex: Perfect. I’ll handle the opening and closing remarks, but someone needs to finalize the Q&A section.

Priya: I can do that once I review the latest client feedback.

Rahul: By the way, did we decide on the software demo flow? The last time, it froze midway.

Alex: Right, I spoke with the tech team. They optimized the backend, and the new build should be stable.

Priya: That’s good news. Should we run one quick test after lunch?

Rahul: Works for me. Around 2 PM?

Alex: Yep, 2 PM it is. Let’s meet in the conference room — we’ll do a dry run of the entire presentation.

Priya: Cool. I’ll bring printouts just in case the projector glitches again.

Rahul: Smart move! Let’s hope everything goes smoothly this time."""
print(sample_text)

print("\n model summary :\n")
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])



In [ ]:
pipe(sample_text,**gen_kwargs)